In [1]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb
import gc
from tqdm.auto import tqdm

In [2]:
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }

In [3]:
pd.options.display.max_columns = 50

In [4]:
h = 28 
max_lags = 70
tr_last = 1913
fday = datetime(2016,4, 25) 
fday

datetime.datetime(2016, 4, 25, 0, 0)

In [5]:
def create_dt(is_train = True, nrows = None, first_day = 1200):
    prices = pd.read_csv("../input/m5-forecasting-accuracy/sell_prices.csv", dtype = PRICE_DTYPES)
    for col, col_dtype in PRICE_DTYPES.items():
        if col_dtype == "category":
            prices[col] = prices[col].cat.codes.astype("int16")
            prices[col] -= prices[col].min()
            
    cal = pd.read_csv("../input/m5-forecasting-accuracy/calendar.csv", dtype = CAL_DTYPES)
    cal["date"] = pd.to_datetime(cal["date"])
    for col, col_dtype in CAL_DTYPES.items():
        if col_dtype == "category":
            cal[col] = cal[col].cat.codes.astype("int16")
            cal[col] -= cal[col].min()
    
    start_day = max(1 if is_train  else tr_last-max_lags, first_day)
    numcols = [f"d_{day}" for day in range(start_day,tr_last+1)]
    catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
    dtype = {numcol:"float32" for numcol in numcols} 
    dtype.update({col: "category" for col in catcols if col != "id"})
    dt = pd.read_csv("../input/m5-forecasting-accuracy/sales_train_validation.csv", 
                     nrows = nrows, usecols = catcols + numcols, dtype = dtype)
    
    for col in catcols:
        if col != "id":
            dt[col] = dt[col].cat.codes.astype("int16")
            dt[col] -= dt[col].min()
    
    if not is_train:
        for day in range(tr_last+1, tr_last+ 28 +1):
            dt[f"d_{day}"] = np.nan
    
    dt = pd.melt(dt,
                  id_vars = catcols,
                  value_vars = [col for col in dt.columns if col.startswith("d_")],
                  var_name = "d",
                  value_name = "sales")
    
    dt = dt.merge(cal, on= "d", copy = False)
    dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
    
    return dt

In [6]:
def create_fea(dt):
    lags = [7, 28]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)

    wins = [7, 28]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())

    
    
    date_features = {
        
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",
#         "ime": "is_month_end",
#         "ims": "is_month_start",
    }
    
#     dt.drop(["d", "wm_yr_wk", "weekday"], axis=1, inplace = True)
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

In [7]:
FIRST_DAY = 1000 # If you want to load all the data set it to '1' -->  Great  memory overflow  risk !
# FIRST_DAY = 1700

In [8]:
%%time

df = create_dt(is_train=True, first_day= FIRST_DAY)
df.shape

CPU times: user 25.6 s, sys: 9.13 s, total: 34.7 s
Wall time: 34.8 s


(26600086, 22)

In [9]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1000,0.0,2013-10-24,11339,4,6,10,2013,0,0,0,0,0.0,0.0,0.0,8.26
1,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1001,2.0,2013-10-25,11339,0,7,10,2013,0,0,0,0,0.0,0.0,0.0,8.26
2,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_1000,0.0,2013-10-24,11339,4,6,10,2013,0,0,0,0,0.0,0.0,0.0,3.97
3,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_1001,0.0,2013-10-25,11339,0,7,10,2013,0,0,0,0,0.0,0.0,0.0,3.97
4,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_1000,1.0,2013-10-24,11339,4,6,10,2013,0,0,0,0,0.0,0.0,0.0,4.64


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26600086 entries, 0 to 26600085
Data columns (total 22 columns):
id              object
item_id         int16
dept_id         int16
store_id        int16
cat_id          int16
state_id        int16
d               object
sales           float32
date            datetime64[ns]
wm_yr_wk        int16
weekday         int16
wday            int16
month           int16
year            int16
event_name_1    int16
event_type_1    int16
event_name_2    int16
event_type_2    int16
snap_CA         float32
snap_TX         float32
snap_WI         float32
sell_price      float32
dtypes: datetime64[ns](1), float32(5), int16(14), object(2)
memory usage: 2.0+ GB


In [11]:
%%time

create_fea(df)
df.shape

CPU times: user 2min 54s, sys: 14.8 s, total: 3min 9s
Wall time: 3min 9s


(26600086, 31)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26600086 entries, 0 to 26600085
Data columns (total 31 columns):
id              object
item_id         int16
dept_id         int16
store_id        int16
cat_id          int16
state_id        int16
d               object
sales           float32
date            datetime64[ns]
wm_yr_wk        int16
weekday         int16
wday            int16
month           int16
year            int16
event_name_1    int16
event_type_1    int16
event_name_2    int16
event_type_2    int16
snap_CA         float32
snap_TX         float32
snap_WI         float32
sell_price      float32
lag_7           float32
lag_28          float32
rmean_7_7       float32
rmean_28_7      float32
rmean_7_28      float32
rmean_28_28     float32
week            int16
quarter         int16
mday            int16
dtypes: datetime64[ns](1), float32(11), int16(17), object(2)
memory usage: 2.7+ GB


In [13]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1000,0.0,2013-10-24,11339,4,6,10,2013,0,0,0,0,0.0,0.0,0.0,8.26,NaN,NaN,NaN,NaN,NaN,NaN,43,4,24
1,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1001,2.0,2013-10-25,11339,0,7,10,2013,0,0,0,0,0.0,0.0,0.0,8.26,NaN,NaN,NaN,NaN,NaN,NaN,43,4,25
2,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_1000,0.0,2013-10-24,11339,4,6,10,2013,0,0,0,0,0.0,0.0,0.0,3.97,NaN,NaN,NaN,NaN,NaN,NaN,43,4,24
3,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_1001,0.0,2013-10-25,11339,0,7,10,2013,0,0,0,0,0.0,0.0,0.0,3.97,NaN,NaN,NaN,NaN,NaN,NaN,43,4,25
4,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_1000,1.0,2013-10-24,11339,4,6,10,2013,0,0,0,0,0.0,0.0,0.0,4.64,NaN,NaN,NaN,NaN,NaN,NaN,43,4,24


In [14]:
df.dropna(inplace = True)
df.shape

(24923136, 31)

In [15]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday"]
train_cols = df.columns[~df.columns.isin(useless_cols)]

In [16]:
train_indexes = df[df['date'] <'2016-03-06'].index
test_indexes = df[df['date'] >= '2016-03-06'].index

In [17]:
train_indexes.shape, test_indexes.shape

((23398991,), (1524145,))

In [18]:
gc.collect()

40

In [19]:
X_train = df.loc[train_indexes, train_cols]
X_test = df.loc[test_indexes, train_cols]
y_train = df.loc[train_indexes, 'sales']
y_test = df.loc[test_indexes, 'sales']

In [20]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((23398991, 25), (1524145, 25), (23398991,), (1524145,))

In [21]:
train_data = lgb.Dataset(X_train, y_train)
valid_data = lgb.Dataset(X_test, y_test)

In [22]:
params = {
        "objective" : "rmse",
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.1,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
#         "lambda_l2" : 0.1,
        "nthread" : -1,
        'verbosity': 1
}
num_iterations = 200

In [23]:
%%time
m_lgb = lgb.train(params, train_data, num_iterations, valid_sets=[train_data, valid_data], verbose_eval=20) 

[20]	training's rmse: 2.31112	valid_1's rmse: 2.13786
[40]	training's rmse: 2.23761	valid_1's rmse: 2.08271
[60]	training's rmse: 2.21182	valid_1's rmse: 2.07648
[80]	training's rmse: 2.19662	valid_1's rmse: 2.07386
[100]	training's rmse: 2.18589	valid_1's rmse: 2.0733
[120]	training's rmse: 2.17741	valid_1's rmse: 2.07298
[140]	training's rmse: 2.16967	valid_1's rmse: 2.07243
[160]	training's rmse: 2.16229	valid_1's rmse: 2.07143
[180]	training's rmse: 2.15671	valid_1's rmse: 2.07119
[200]	training's rmse: 2.15084	valid_1's rmse: 2.07052
CPU times: user 18min 59s, sys: 7.59 s, total: 19min 7s
Wall time: 5min 24s


In [24]:
m_lgb.save_model("baseline_model.lgb")

In [25]:
%%time

# alphas = [1.035, 1.03, 1.025, 1.02]
alphas = [1.035]
weights = [1/len(alphas)]*len(alphas)
sub = 0.

for icount, (alpha, weight) in tqdm(enumerate(zip(alphas, weights))):

    te = create_dt(False)
    cols = [f"F{i}" for i in range(1,29)]

    for tdelta in tqdm(range(0, 28)): 
        day = fday + timedelta(days=tdelta)
        print(icount, day)
        tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()
        create_fea(tst)
        tst = tst.loc[tst.date == day , train_cols]
        te.loc[te.date == day, "sales"] = alpha*m_lgb.predict(tst) # magic multiplier by kyakovlev



    te_sub = te.loc[te.date >= fday, ["id", "sales"]].copy()
#     te_sub.loc[te.date >= fday+ timedelta(days=h), "id"] = te_sub.loc[te.date >= fday+timedelta(days=h), 
#                                                                           "id"].str.replace("validation$", "evaluation")
    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]
    te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()
    te_sub.fillna(0., inplace = True)
    te_sub.sort_values("id", inplace = True)
    te_sub.reset_index(drop=True, inplace = True)
#     te_sub.to_csv("submission.csv",index=False)
    if icount == 0 :
        sub = te_sub
        sub[cols] *= weight
    else:
        sub[cols] += te_sub[cols]*weight
    print(icount, alpha, weight)

0 2016-04-25 00:00:00
0 2016-04-26 00:00:00
0 2016-04-27 00:00:00
0 2016-04-28 00:00:00
0 2016-04-29 00:00:00
0 2016-04-30 00:00:00
0 2016-05-01 00:00:00
0 2016-05-02 00:00:00
0 2016-05-03 00:00:00
0 2016-05-04 00:00:00
0 2016-05-05 00:00:00
0 2016-05-06 00:00:00
0 2016-05-07 00:00:00
0 2016-05-08 00:00:00
0 2016-05-09 00:00:00
0 2016-05-10 00:00:00
0 2016-05-11 00:00:00
0 2016-05-12 00:00:00
0 2016-05-13 00:00:00
0 2016-05-14 00:00:00
0 2016-05-15 00:00:00
0 2016-05-16 00:00:00
0 2016-05-17 00:00:00
0 2016-05-18 00:00:00
0 2016-05-19 00:00:00
0 2016-05-20 00:00:00
0 2016-05-21 00:00:00
0 2016-05-22 00:00:00

0 1.035 1.0

CPU times: user 52min 30s, sys: 22.5 s, total: 52min 53s
Wall time: 52min 44s


In [26]:
sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv("submission.csv",index=False)

In [27]:
sub.head(10)

F,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,1.220401,1.022107,1.022107,0.980644,1.091866,1.373423,1.396068,1.129386,1.043009,1.042349,1.050986,1.089264,1.467123,1.461884,1.121312,1.013904,0.984811,1.020016,1.079722,1.353979,1.312788,1.078930,1.017784,1.005461,0.985110,1.110484,1.328315,1.418130
1,FOODS_1_001_CA_2_validation,1.086177,1.032819,1.091845,1.560644,1.420348,1.850611,1.964511,1.209571,1.153643,1.123077,1.117878,1.182754,1.646023,1.620445,1.326257,1.229315,1.180145,1.235427,1.290365,1.636615,1.884136,1.364705,1.272993,1.202833,1.269562,1.424149,1.910736,1.682782
2,FOODS_1_001_CA_3_validation,1.236535,1.188104,1.127941,1.127143,1.136823,1.292364,0.921995,0.841675,0.843688,0.855397,0.911982,1.051668,1.313042,1.327876,1.163553,1.076636,1.082748,1.082748,1.131125,1.314640,1.435624,1.029411,1.027413,1.062252,1.041902,1.130108,1.377278,1.114394
3,FOODS_1_001_CA_4_validation,0.372258,0.267377,0.239337,0.239337,0.340385,0.401809,0.532363,0.375386,0.392720,0.360612,0.353362,0.391140,0.469168,0.404259,0.401854,0.360167,0.396514,0.396514,0.451012,0.501903,0.506765,0.452375,0.403153,0.396284,0.375933,0.429858,0.533450,0.537017
4,FOODS_1_001_TX_1_validation,0.164059,0.141712,0.130408,0.130408,0.193225,0.190810,0.196110,0.302352,0.243339,0.228966,0.226958,0.294879,0.309762,0.291439,0.280592,0.237199,0.242934,0.257688,0.315754,0.323975,0.322777,0.277408,0.242617,0.238513,0.218163,0.272449,0.340695,0.340695
5,FOODS_1_001_TX_2_validation,0.491979,0.459049,0.459049,0.379025,0.359271,0.450217,0.438405,0.456909,0.446830,0.422137,0.428597,0.474684,0.563669,0.525025,0.502087,0.439837,0.449760,0.469770,0.504699,0.579721,0.544407,0.510030,0.460808,0.456704,0.436354,0.461007,0.517007,0.524423
6,FOODS_1_001_TX_3_validation,0.495291,0.405290,0.405290,0.518251,0.506155,0.538412,0.530583,0.515406,0.544245,0.519553,0.510134,0.564250,0.629977,0.562074,0.575812,0.494580,0.541179,0.541179,0.562569,0.634090,0.602277,0.527724,0.478503,0.511074,0.454048,0.545010,0.608425,0.571750
7,FOODS_1_001_WI_1_validation,0.311554,0.419876,0.361331,0.361331,0.354335,1.337138,0.994168,0.840440,0.791987,0.760737,0.813275,0.853466,0.688345,0.695071,0.651617,0.608594,0.618517,0.638215,0.703697,0.852642,0.854857,0.677588,0.676367,0.672262,0.651912,0.689160,1.029191,0.925088
8,FOODS_1_001_WI_2_validation,0.354918,0.372972,0.374890,0.348263,0.378752,0.404682,0.425293,0.488346,0.468636,0.369256,0.418813,0.549822,0.518399,0.531919,0.531584,0.442326,0.485919,0.485919,0.495868,0.563477,0.608515,0.445670,0.403111,0.332087,0.354409,0.391001,0.509271,0.509271
9,FOODS_1_001_WI_3_validation,0.210357,0.182217,0.154177,0.154177,0.191002,0.587564,0.424642,0.478146,0.458436,0.403513,0.429610,0.502121,0.389414,0.450849,0.446591,0.378018,0.421853,0.421853,0.431802,0.545998,0.550860,0.391785,0.393047,0.378763,0.357165,0.411452,0.489445,0.471410


In [28]:
sub.id.nunique(), sub["id"].str.contains("validation$").sum()

(60980, 30490)

In [29]:
sub.shape

(60980, 29)